# Visão Computacional - Projeto 2-1

### Bruna Kimura

In [2]:
%matplotlib inline

import cv2 as cv2
import numpy as np     
import matplotlib.pyplot as plt

import time

### 1. Detecção de alguns pontos interessantes na imagem

In [ ]:
captura = cv2.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv2.CAP_PROP_BUFFERSIZE, 1)

while(1):
    ret, frame = captura.read()
    
    # Pressione ESC para sair do loop
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    time.sleep(0.1)
    
    # Converte para Grayscale
    bw = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Parametriza a funcao do OpenCV
    params = dict( maxCorners = 100,
                   qualityLevel = 0.3,
                   minDistance = 7,
                   blockSize = 7 )

    # Funcao que retorna uma lista de pontos
    pts = cv2.goodFeaturesToTrack(bw, mask = None, **params)
    
    # Gera cores de forma aleatória
    color = np.random.randint(0,255,(100,3))

    # Insere uma marcação em cada ponto:
    for i,pt in enumerate(pts):
        a,b = pt.ravel()
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)

    cv2.imshow("Video", frame)

captura.release()
cv2.destroyAllWindows()

### 2. Rastreamento dos pontos (apenas em dois frames)

In [31]:
captura = cv2.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv2.CAP_PROP_BUFFERSIZE, 1)

# Parametriza a funcao do OpenCV
dt_params = dict( maxCorners = 100,
                  qualityLevel = 0.3,
                  minDistance = 7,
                  blockSize = 7 )

# Parametriza o Lucas-Kanade
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Gera cores de forma aleatória
color = np.random.randint(0,255,(100,3))

ret, frame = captura.read()
previous = frame
previous_gray = cv2.cvtColor(previous, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(previous_gray, mask = None, **dt_params)

# Cria uma máscara para imprimir o rastro.
mask = np.zeros_like(previous)

m_x = 0
m_y = 0


while(1):
    ret, frame = captura.read()
    
    # Pressione ESC para sair do loop
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    time.sleep(0.1)
    
    actual = frame
    actual_gray = cv2.cvtColor(actual, cv2.COLOR_BGR2GRAY)
    
    # Calcula o Fluxo Otico
    p1, st, err = cv2.calcOpticalFlowPyrLK(previous_gray, actual_gray, p0, None, **lk_params)
    
    # Seleciona somente os melhores pontos
    good_new = p1[st==1]
    good_old = p0[st==1]
    
    
    
    # Desenha as trilhas para cada ponto em p1 e p0
    for i,(new, old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), [0,0,255], 2)
        actual = cv2.circle(actual,(a,b),5,color[i].tolist(),-1)
        
    
    # diferença entre os pontos antigos e os atuais
    dif = np.subtract(good_old, good_new)
    
    print(dif.shape)
    
    x = dif[:,0]
    y = dif[:,1]
    
    len_x = len(x)
    len_y = len(y)
    
    soma_x = 0
    soma_y = 0
    
    for e in x:
        soma_x += e
        
    for e in y:
        soma_y += e
        
    m_x += (soma_x/len_x)
    m_y += (soma_y/len_y)
    
    translation_matrix = np.float32([[1,0,int(m_x)], [0,1,int(m_y)]])
    
    
        
    #a primeira coluna são os x's e a segunda o y', precisa tirar as médias p saber o quanto precisa fazer resize
    #resize da imagem
       
    
    img = cv2.add(actual, mask)
    
    num_rows, num_cols = img.shape[:2]
    img_translation = cv2.warpAffine(img, translation_matrix, (num_cols, num_rows))
    
    cv2.imshow('Translation', img_translation)
    
    # Now update the previous frame and previous points
    previous_gray = actual_gray.copy()
    p0 = good_new.reshape(-1,1,2)

    
captura.release()
cv2.destroyAllWindows()


(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)


### Dense


In [45]:
captura = cv2.VideoCapture(0)
ret, frame1 = captura.read()

img = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255

height, width = img.shape[:2]
prvs = cv2.resize(img,(2*width, 2*height), interpolation = cv2.INTER_CUBIC)

m_x = 0
m_y = 0

while(1):
    ret, frame2 = captura.read()
    next1 = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    
    flow = cv2.calcOpticalFlowFarneback(prvs,next1, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    print(flow.shape)
    
    
    x = flow[:,:,0]
    y = flow[:,:,1]
    
    print('x:', x)
    print('y:', y)
    
    len_x = len(x)
    len_y = len(y)
    
    soma_x = 0
    soma_y = 0
    
    for e in x:
        soma_x += e
        
    for e in y:
        soma_y += e
        
    m_x += (soma_x/len_x)
    m_y += (soma_y/len_y)
    
    translation_matrix = np.float32([[1,0,m_x], [0,1,m_y]])
    
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
    
    num_rows, num_cols = bgr.shape[:2]
    img_translation = cv2.warpAffine(bgr, np.uint8(translation_matrix), (num_cols, num_rows))
    
    cv2.imshow('Translation', img_translation)
    
    
    
    cv2.imshow('frame2',bgr)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    prvs = next1
    
captura.release()
cv2.destroyAllWindows()

error: OpenCV(3.4.2) /io/opencv/modules/video/src/optflowgf.cpp:1114: error: (-215:Assertion failed) prev0.size() == next0.size() && prev0.channels() == next0.channels() && prev0.channels() == 1 && pyrScale_ < 1 in function 'calc'


In [46]:
captura.release()